# Normalización de Campos del Corpus CMV
Este notebook implementa la normalización de campos del corpus CMV. Toma como entrada el archivo `corpus_cmv.csv` (generado tras el parsing de `.ushay`) y aplica reglas de normalización para dejar el corpus listo para evaluación, visualización o ingestión posterior en un sistema RAG.

## 1. Cargar corpus original

In [4]:

import pandas as pd

# Ruta del corpus base
INPUT_PATH = "../../data_curated/corpus_estructurado/corpus_cmv.csv"
df = pd.read_csv(INPUT_PATH, dtype=str)
print(f"Registros: {len(df)}, Columnas: {df.shape[1]}")
df.head(2)


Registros: 2, Columnas: 265


,file,CP,CPP,DO,VERSION,ID,RUC,LOGO_PATH,NOMBRE_ENTID_CONTRAT,PROVINCIA_COD,...,TIPO_REGIMEN_COD,TIPO_REGIMEN,TIPO_PRESUPUESTO_COD,TIPO_PRESUPUESTO,TIPO_PRODUCTO_COD,TIPO_PRODUCTO,PROCEDIMIENTO_COD,PROCEDIMIENTO,VALOR_PAC,PARTIDA_PRESUPUESTARIA
0,oferta_pli_cotobr.ushay,d49a8f53c5feb3a8077827b9d2a8a54c,COTO-EPN-029-2023,1001581055001,1.6.0,8,1760005620001,1760005620001principal_color.jpg,ESCUELA POLITÉCNICA NACIONAL,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,contratacion_pli_cotobr.ushay,d9f4df1b886038572eb272e5d62c45a4,NaN,NaN,1.2,14,0960006180001,escudo.jpg,GAD MUNICIPAL DEL CANTON NOBOL,09,...,7000,COMUN,7002,PROYECTO DE INVERSIÓN,0,NO APLICA,9,COTIZACION,495930.26,0


## 2. Limpieza básica de texto

In [5]:

df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
df.replace({"": None, "NA": None, "na": None, "null": None, "None": None}, inplace=True)


## 3. Normalización de fechas (formato ISO-8601)

In [ ]:
# 1) identificar columnas de fecha por nombre (búsqueda parcial, case-insensitive)
fecha_cols = [c for c in df.columns if "FECHA" in c.upper()]

print(f"Columnas detectadas con 'FECHA': {fecha_cols}")

def normalize_any_date(x):
    """Conversión ligera a fecha:
    - intenta parsear con pandas (errors='coerce')
    - si falla, reintenta con dayfirst=True
    - devuelve 'YYYY-MM-DD' o None si no se pudo
    """
    if pd.isna(x):
        return None
    s = str(x).strip()
    s = s.replace("\\", "/").replace(".", "/")
    
    # intento 1: parseo libre
    dt = pd.to_datetime(s, errors="coerce")
    # intento 2: asumiendo día/mes primero (útil en datos locales)
    if pd.isna(dt):
        dt = pd.to_datetime(s, errors="coerce", dayfirst=True)
    if pd.isna(dt):
        return None
    # retornar solo fecha en ISO
    return dt.date().isoformat()

# 2) crear columnas *_NORMALIZADA para cada columna de fecha
for col in fecha_cols:
    out_col = f"{col}_NORMALIZADA"
    df[out_col] = df[col].apply(normalize_any_date)
    # mini-resumen por columna
    ok = df[out_col].notna().sum()
    total = len(df)
    print(f"→ {out_col}: {ok}/{total} filas convertidas")


df[[c for c in df.columns if c.endswith("_NORMALIZADA")]].head()



Columnas detectadas con 'FECHA': ['FECHA', 'DIAS_FECHALIMITECONVALIDACION', 'ES_FECHA_ENTREGA', 'FECHA_FIRMA_OFERTA', 'TIPO_FECHA_CRONOGRAMA', 'FECHA_FABRICACION', 'FECHA_RECEPCION', 'TIPO_FECHA', 'FECHA_NACIMIENTO_CP', 'FECHA_GRADUACION_CP', 'FECHA_OBTEN_TITUL_CP']
→ FECHA_NORMALIZADA: 2/2 filas convertidas
→ DIAS_FECHALIMITECONVALIDACION_NORMALIZADA: 0/2 filas convertidas
→ ES_FECHA_ENTREGA_NORMALIZADA: 0/2 filas convertidas
→ FECHA_FIRMA_OFERTA_NORMALIZADA: 1/2 filas convertidas
→ TIPO_FECHA_CRONOGRAMA_NORMALIZADA: 0/2 filas convertidas
→ FECHA_FABRICACION_NORMALIZADA: 1/2 filas convertidas
→ FECHA_RECEPCION_NORMALIZADA: 1/2 filas convertidas
→ TIPO_FECHA_NORMALIZADA: 0/2 filas convertidas
→ FECHA_NACIMIENTO_CP_NORMALIZADA: 1/2 filas convertidas
→ FECHA_GRADUACION_CP_NORMALIZADA: 1/2 filas convertidas
→ FECHA_OBTEN_TITUL_CP_NORMALIZADA: 1/2 filas convertidas


,FECHA_NORMALIZADA,DIAS_FECHALIMITECONVALIDACION_NORMALIZADA,ES_FECHA_ENTREGA_NORMALIZADA,FECHA_FIRMA_OFERTA_NORMALIZADA,TIPO_FECHA_CRONOGRAMA_NORMALIZADA,FECHA_FABRICACION_NORMALIZADA,FECHA_RECEPCION_NORMALIZADA,TIPO_FECHA_NORMALIZADA,FECHA_NACIMIENTO_CP_NORMALIZADA,FECHA_GRADUACION_CP_NORMALIZADA,FECHA_OBTEN_TITUL_CP_NORMALIZADA
0,2023-07-14,None,None,2023-07-08,None,2023-01-01,2020-01-14,None,1986-08-26,2012-03-26,2012-03-26
1,2023-07-20,None,None,None,None,None,None,None,None,None,None


## 4. Homologación de categorías

In [7]:

# Ejemplo: normalizar tipo de persona
mapeo_tipo_persona = {
    "NATURAL": "Natural", "NAT": "N",
    "JURIDICA": "Jurídica", "JUR": "J"
}
if "TIPO_PERSONA" in df.columns:
    df["TIPO_PERSONA"] = df["TIPO_PERSONA"].replace(mapeo_tipo_persona)


## 5. Conversión de montos a float

In [8]:

def limpiar_monto(valor):
    if pd.isna(valor):
        return None
    v = str(valor).replace("$", "").replace(",", "").strip()
    try:
        return float(v)
    except:
        return None

for col in ["PRESUPUESTO_REFERENCIAL_NUMEROS", "VALOR_OFERTADO", "VALOR_CONTRATO", "TOTAL"]:
    if col in df.columns:
        df[col + "_USD"] = df[col].apply(limpiar_monto)


## 6. Enriquecimiento de campos derivados

In [9]:

# Año de publicación
if "FECHA_NORMALIZADA" in df.columns:
    df["ANIO"] = df["FECHA_NORMALIZADA"].str[:4]
    print(df["ANIO"].value_counts())


ANIO
2023    2
Name: count, dtype: int64


## 7. Guardar corpus normalizado

In [11]:

OUTPUT_PATH = "../../data_curated/corpus_estructurado/corpus_cmv.csv"
df.to_csv(OUTPUT_PATH, index=False)
print(f"Archivo exportado a: {OUTPUT_PATH}")


Archivo exportado a: ../../data_curated/corpus_estructurado/corpus_cmv.csv
